# Tarea 1 Datos Masivos 1
## Acosta Imandt Daniel

cargar los datos a databricks y cargarlos a tu libreta como RDD y como dataFrame.

In [0]:
#Carga de la biblioteca
import pyspark
# Carga funciones extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('primeros_pasos').getOrCreate()

In [0]:

#cargamos los datos como RDD
RDD_crudos = spark.sparkContext.textFile('/FileStore/tables/players_15.csv')

In [0]:
#Mcargamos los datos como dataFrame
df_fifa = spark.read.csv('/FileStore/tables/players_15.csv', header = True, inferSchema = True)

#Dos operaciones con RDD

In [0]:
for renglon in RDD_crudos.take(2):
  print(renglon)

sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defe

In [0]:
# Primero lo ordenamos un poco
# Guarda el primer elemento 
columnas = RDD_crudos.take(1)[0]
# Guarda en todos los elementos de RDD_crudos menos el primero correspondiente a las columnas
RDD_crudos = RDD_crudos.filter(lambda renglon : renglon != columnas)

def separar_campos(renglon):
    renglon_separado = renglon.split(',')
    # Extrae los campos que queremos
    nombre = str(renglon_separado[2])
    edad = int(renglon_separado[4])
    estatura = int(renglon_separado[6])
    peso = int(renglon_separado[7])
    nacionalidad = str(renglon_separado[8])
    equipo = str(renglon_separado[9])
    potencial = int(renglon_separado[11])
    return (nombre, edad, estatura, peso, nacionalidad, equipo, potencial)

In [0]:
#aplicamos la función a los datos

rdd_fifa = RDD_crudos.map(separar_campos)
# Imprimimos una muestra de los datos
for renglon in rdd_fifa.take(5):
  print(renglon)

('L. Messi', 27, 169, 67, 'Argentina', 'FC Barcelona', 95)
('Cristiano Ronaldo', 29, 185, 80, 'Portugal', 'Real Madrid', 92)
('A. Robben', 30, 180, 80, 'Netherlands', 'FC Bayern München', 90)
('Z. Ibrahimović', 32, 195, 95, 'Sweden', 'Paris Saint-Germain', 90)
('M. Neuer', 28, 193, 92, 'Germany', 'FC Bayern München', 90)


### Primero vamos a querer ver cuantos jugadores existen por club

In [0]:
# Nos quedamos solo con la entrada corresponiente al club
rdd_club = rdd_fifa.map(lambda renglon: (renglon[5], 1))

In [0]:
for i in rdd_club.take(5):
    print(i)

('FC Barcelona', 1)
('Real Madrid', 1)
('FC Bayern München', 1)
('Paris Saint-Germain', 1)
('FC Bayern München', 1)


In [0]:
#Usando los clubes como llaves y la transformación llamada reduceByKey, sumaremos todos los valores para cada país.
rdd_club = rdd_club.reduceByKey(lambda val_1 , val_2: val_1 + val_2)

In [0]:
#Para visualizar los resultados, usaremos la acción collect(). Esta acción devuelve todo el contenido del RDD. 

resultados = rdd_club.collect()

for renglon in resultados[:10]:
  print(renglon)

('FC Bayern München', 29)
('Chelsea', 33)
('Manchester United', 33)
('Manchester City', 33)
('Borussia Dortmund', 33)
('Atlético Madrid', 29)
('Galatasaray SK', 28)
('Zenit St. Petersburg', 30)
('Lazio', 33)
('Milan', 27)


In [0]:
# Pedimos que ordene por total de jugadores
rdd_club = rdd_club.sortBy(lambda renglon: renglon[1], ascending = False)

for renglon in rdd_club.take(10):
  print(renglon)

('Chelsea', 33)
('Manchester United', 33)
('Manchester City', 33)
('Borussia Dortmund', 33)
('Lazio', 33)
('Athletic Club de Bilbao', 33)
('Sevilla FC', 33)
('Southampton', 33)
('Olympique Lyonnais', 33)
('RCD Espanyol', 33)


Por lo que ya tenemos el número de jugadores por club, al parecer el máximo es 33 jugadores.

### Ahora vamos a querer ver cual es la suma de potencial de los jugadores por club para encontrar el club más poderoso

In [0]:
# Nos quedamos solo con la entrada corresponiente al club
rdd_pot= rdd_fifa.map(lambda renglon: (renglon[5],renglon[6]))

In [0]:
for i in rdd_pot.take(5):
    print(i)

('FC Barcelona', 95)
('Real Madrid', 92)
('FC Bayern München', 90)
('Paris Saint-Germain', 90)
('FC Bayern München', 90)


In [0]:
#Usando los clubes como llaves y la transformación llamada reduceByKey, sumaremos todos los valores para cada país.
rdd_pot = rdd_pot.reduceByKey(lambda val_1 , val_2: val_1 + val_2)

In [0]:
#Para visualizar los resultados, usaremos la acción collect(). Esta acción devuelve todo el contenido del RDD. 

resultados_pot = rdd_pot.collect()

for renglon in resultados_pot[:10]:
  print(renglon)

('FC Bayern München', 2419)
('Chelsea', 2639)
('Manchester United', 2605)
('Manchester City', 2612)
('Borussia Dortmund', 2611)
('Atlético Madrid', 2276)
('Galatasaray SK', 2156)
('Zenit St. Petersburg', 2290)
('Lazio', 2544)
('Milan', 2146)


In [0]:
# Pedimos que ordene por total de jugadores
rdd_pot= rdd_pot.sortBy(lambda renglon: renglon[1], ascending = False)

for renglon in rdd_pot.take(10):
  print(renglon)

('Real Madrid', 2653)
('Juventus', 2641)
('Chelsea', 2639)
('Arsenal', 2615)
('Manchester City', 2612)
('Borussia Dortmund', 2611)
('Manchester United', 2605)
('Tottenham Hotspur', 2598)
('Liverpool', 2596)
('FC Schalke 04', 2582)


Por lo tanto llegamos a que el club más poderoso es el Real Madrid

#Dos operaciones con dataFrame

In [0]:
df_fifa

Out[78]: DataFrame[sofifa_id: int, player_url: string, short_name: string, long_name: string, age: int, dob: string, height_cm: int, weight_kg: int, nationality: string, club: string, overall: int, potential: int, value_eur: int, wage_eur: int, player_positions: string, preferred_foot: string, international_reputation: int, weak_foot: int, skill_moves: int, work_rate: string, body_type: string, real_face: string, release_clause_eur: string, player_tags: string, team_position: string, team_jersey_number: int, loaned_from: string, joined: string, contract_valid_until: int, nation_position: string, nation_jersey_number: int, pace: int, shooting: int, passing: int, dribbling: int, defending: int, physic: int, gk_diving: int, gk_handling: int, gk_kicking: int, gk_reflexes: int, gk_speed: int, gk_positioning: int, player_traits: string, attacking_crossing: string, attacking_finishing: string, attacking_heading_accuracy: string, attacking_short_passing: string, attacking_volleys: string, skil

In [0]:
#Semejante a pandas, podemos seleccionar un conjunto de columnas
df_fifa = df_fifa.select('short_name','age', 'height_cm', 'weight_kg', 'nationality', 'club', 'potential', 'preferred_foot')

#dada la estructrua que el DataFrame da a los datos, podemos hacer referencia a las columnas por su nombre en el archivo CSV.

In [0]:
df_fifa.show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|potential|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|         L. Messi| 27|      169|       67|  Argentina|       FC Barcelona|       95|          Left|
|Cristiano Ronaldo| 29|      185|       80|   Portugal|        Real Madrid|       92|         Right|
|        A. Robben| 30|      180|       80|Netherlands|  FC Bayern München|       90|          Left|
|   Z. Ibrahimović| 32|      195|       95|     Sweden|Paris Saint-Germain|       90|         Right|
|         M. Neuer| 28|      193|       92|    Germany|  FC Bayern München|       90|         Right|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



###Vamos a querer ver los 5 jugadores con más potencial menores a 22 años

In [0]:
#buscando a los jugadores con más potencial menores a 22 años
df_fifa.where((df_fifa.age < 22) ).orderBy('potential', ascending = False).show(5)

+----------+---+---------+---------+-----------+--------------------+---------+--------------+
|short_name|age|height_cm|weight_kg|nationality|                club|potential|preferred_foot|
+----------+---+---------+---------+-----------+--------------------+---------+--------------+
|  P. Pogba| 21|      188|       80|     France|            Juventus|       89|         Right|
| J. Iturbe| 21|      172|       68|  Argentina|                Roma|       88|          Left|
| R. Varane| 21|      191|       78|     France|         Real Madrid|       88|         Right|
|A. Laporte| 20|      189|       85|     France|Athletic Club de ...|       88|          Left|
|  M. Depay| 20|      176|       78|Netherlands|                 PSV|       88|         Right|
+----------+---+---------+---------+-----------+--------------------+---------+--------------+
only showing top 5 rows



Por lo tanto el mejor jugador menor a 22 años es Pogba

### Queremos ver las nacionalidades con el promedio de jugadores más viejos

In [0]:
df_fifa.groupBy('nationality').agg(mean('age').alias('edad_promedio')).orderBy('edad_promedio', ascending =False).show(5) 

+-----------------+------------------+
|      nationality|     edad_promedio|
+-----------------+------------------+
|          Grenada|              36.0|
|            Malta|34.333333333333336|
|          Bahrain|              33.0|
|             Oman|              32.0|
|         Barbados|              30.5|
|           Kuwait|              30.0|
|        Palestine|              29.0|
|            Syria|28.666666666666668|
|          Belarus|            28.625|
|        Gibraltar|              28.0|
|             Cuba|              28.0|
|          Jamaica|27.871794871794872|
|          Armenia|27.833333333333332|
|           Panama|27.636363636363637|
|         Slovakia|27.568181818181817|
|Trinidad & Tobago|              27.5|
|         Paraguay|27.430232558139537|
|         Honduras| 27.41176470588235|
|       Uzbekistan|              27.4|
|          Moldova|27.285714285714285|
|         Bulgaria| 27.28205128205128|
|             Togo|             27.25|
|        Guatemala|      

Por lo que vemos que Granada tiene los jugadores más viejos

#Consulta SQL

###Vamos a  ver el promedio de potencial de los jugadores para los 5 mejores clubs

In [0]:
#Por ejemplo, si quisieramos consultar de nuevo cuales son los 5 equipos más caros, solo que esta vez haciendo una consulta SQL, haríamos los siguiente

# Le dice a spark que guarde este DataFrame como una tabla SQL
df_fifa.registerTempTable('FIFA15')

In [0]:
%sql 
/
SELECT club, MEAN(potential) AS valor_total 
  FROM FIFA15
  GROUP BY club
  ORDER BY valor_total DESC
  limit 5

club,valor_total
FC Barcelona,83.45833333333333
FC Bayern München,83.41379310344827
Paris Saint-Germain,81.45833333333333
Austria,81.0
Real Madrid,80.39393939393939
